# Lista 01

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
# from scipy.spatial.distance import pdist, squareform

# from scipy.stats import norm
# from itertools import combinations

import statsmodels.api as sm

### Using dataset from Peng's Book

https://dataverse.harvard.edu/file.xhtml?fileId=7440281&version=3.0

In [2]:
lalonde = pd.read_csv("cps1re74.csv", sep=r"\s+|;|:", engine="python")
lalonde["u74"] = (lalonde["re74"] == 0).astype(int)
lalonde["u75"] = (lalonde["re75"] == 0).astype(int)

new_column_names = [
    "treat",
    "age",
    "educ",
    "black",
    "hispan",
    "married",
    "nodegree",
    "re74",
    "re75",
    "u74",
    "u75",
    "re78",
]
lalonde = lalonde.reindex(columns=new_column_names)

lalonde.describe()

,treat,age,educ,black,hispan,married,nodegree,re74,re75,u74,u75,re78
count,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000,16177.000000
mean,0.011436,33.140508,12.008283,0.082339,0.071892,0.705755,0.300550,13887.722590,13504.038995,0.126352,0.114916,14758.358701
std,0.106329,11.036508,2.868005,0.274889,0.258317,0.455717,0.458511,9622.151843,9302.829970,0.332256,0.318931,9681.124686
min,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,24.000000,11.000000,0.000000,0.000000,0.000000,0.000000,4080.000000,4100.000000,0.000000,0.000000,5490.000000
50%,0.000000,31.000000,12.000000,0.000000,0.000000,1.000000,0.000000,14900.000000,14400.000000,0.000000,0.000000,16200.000000
75%,0.000000,42.000000,13.000000,0.000000,0.000000,1.000000,1.000000,23500.000000,22800.000000,0.000000,0.000000,25600.000000
max,1.000000,55.000000,18.000000,1.000000,1.000000,1.000000,1.000000,35040.070000,25200.000000,1.000000,1.000000,60307.930000


In [10]:
y = lalonde["re78"]
z = lalonde["treat"]
x = lalonde[new_column_names[1:-1]]

print(f"Treatment samples: {z[z==1].count()}")
print(f"Control samples: {z[z==0].count()}")

Treatment samples: 185
Control samples: 15992


In [55]:
scaler = MinMaxScaler()
x_scaled = x.copy()
x_scaled[new_column_names[1:-1]] = scaler.fit_transform(x)

def distance(xi, xj):
    xi_ = xi.values
    xj_ = xj.values
    return np.sqrt(np.sum((xi_ - xj_) ** 2))

def get_M_closest(x_treat, x_controls, m):
    d_ = {}
    for index, row in x_controls.iterrows():
        d_[index] = distance(x_treat, row)
    d_sorted = sorted(d_.items(), key=lambda x: x[1])
    m_closest = [key for key, _ in d_sorted[:m]]
    return m_closest

In [67]:
m = 5
control = []
for index in x[z==1].index:
    closest = get_M_closest(x_scaled[z==1].loc[index], x_scaled[z==0], m)
    control.extend(closest)

In [135]:
control_ = list(set(control))
print(f'Matching samples: {len(control_)}')

all_samples = list(z[z==1].index)
all_samples.extend(control_)
print(f'All samples: {len(all_samples)}')

k = dict(Counter(all_samples))

Matching samples: 385
All samples: 570


In [129]:
model = {}
model_fit = {}
mu = {}

model[0] = sm.OLS(y.loc[control_], sm.add_constant((x.loc[control_])))
model_fit[0] = model[0].fit()
mu[0] = model_fit[0].predict(sm.add_constant(x.loc[all_samples]))

model[1] = sm.OLS(y[z==1], sm.add_constant((x[z==1])))
model_fit[1] = model[1].fit()
mu[1] = model_fit[1].predict(sm.add_constant(x.loc[all_samples]))

In [112]:
model_fit[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   re78   R-squared:                       0.315
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     17.17
Date:                Thu, 06 Jun 2024   Prob (F-statistic):           1.15e-25
Time:                        07:06:29   Log-Likelihood:                -3874.6
No. Observations:                 385   AIC:                             7771.
Df Residuals:                     374   BIC:                             7815.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6703.4692   2944.165      2.277      0.023     914.277    1.25e+04
age         -154.8510     39.051     -3.965      0.000    -231.638     -78.064
educ         113.2114    192.029      0.590      0.556    -264.381     490.803
black        275.0731    765.841      0.359      0.720   -1230.821    1780.967
hispan       542.6376   1118.086      0.485      0.628   -1655.886    2741.161
married      302.6570    734.037      0.412      0.680   -1140.700    1746.014
nodegree    -512.4528    909.694     -0.563      0.574   -2301.209    1276.303
re74           0.0488      0.103      0.473      0.636      -0.154       0.252
re75           0.7789      0.121      6.454      0.000       0.542       1.016
u74         -983.0966    967.435     -1.016      0.310   -2885.391     919.198
u75          604.6279    915.118      0.661      0.509   -1194.793    2404.049
==============================================================================
Omnibus:                       50.588   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.565
Skew:                           0.882   Prob(JB):                     2.88e-16
Kurtosis:                       4.160   Cond. No.                     8.11e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [113]:
model_fit[1].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   re78   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     2.002
Date:                Thu, 06 Jun 2024   Prob (F-statistic):             0.0357
Time:                        07:06:37   Log-Likelihood:                -1911.5
No. Observations:                 185   AIC:                             3845.
Df Residuals:                     174   BIC:                             3880.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3128.7565   5925.984     -0.528      0.598   -1.48e+04    8567.308
age           57.9829     85.977      0.674      0.501    -111.710     227.675
educ         548.2293    386.988      1.417      0.158    -215.565    1312.024
black       -590.4062   1947.146     -0.303      0.762   -4433.471    3252.659
hispan      1262.0935   3003.002      0.420      0.675   -4664.906    7189.093
married      935.6978   1553.947      0.602      0.548   -2131.314    4002.710
nodegree    -770.6822   1771.869     -0.435      0.664   -4267.804    2726.440
re74           0.2925      0.180      1.627      0.105      -0.062       0.647
re75           0.0119      0.260      0.046      0.964      -0.502       0.526
u74         6920.8818   2170.219      3.189      0.002    2637.540    1.12e+04
u75        -4052.4794   1849.011     -2.192      0.030   -7701.857    -403.102
==============================================================================
Omnibus:                      120.342   Durbin-Watson:                   2.086
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1066.508
Skew:                           2.341   Prob(JB):                    2.57e-232
Kurtosis:                      13.790   Cond. No.                     6.52e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.52e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [155]:
def phi_i(i):
    x = mu[1].loc[i] - mu[0].loc[i]
    x += (2*z.loc[i] - 1) * (1 + k[i]/m) * (y.loc[i] - mu[z.loc[i]].loc[i])
    return x

In [157]:
phi = [phi_i(i) for i in all_samples]

In [160]:
tau = sum(phi) / len(phi)
print(f'tau = {tau}')

tau = 964.5178112116042


In [162]:
v_i_2 = [(p - tau)**2 for p in phi]
v = sum(v_i_2) / len(phi)**2
print(f'Sdt_dev = {np.sqrt(v)}')

Sdt_dev = 362.75610168930297
